In [ ]:
!pip install apache-beam[gcp] pydicom nibabel google-cloud-storage

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 98.1 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod-1.7-cp310-cp310-linux_x86_64.whl size=23090 sha256=1616d34f6c26d949fb0c2734ba07947596a4f81e1aedb52c61517169cd69a633
  Stored in directory: /home/jupyter/.cache/pip/wheels/85/4c/07/72215c529bd59d67e3dac29711

In [5]:
import apache_beam as beam
import pydicom
import nibabel as nib
from apache_beam.options.pipeline_options import PipelineOptions

def nifti_to_dicom(input_path, output_path):
    """Converts a single NIfTI file to a DICOM series."""

    # Load the NIfTI file
    nifti_img = nib.load(input_path)
    nifti_data = nifti_img.get_fdata()

    # Get NIfTI header information (important for DICOM metadata)
    header = nifti_img.header

    # Create a DICOM dataset (you'll need to populate metadata carefully)
    dicom_ds = pydicom.Dataset()
    # ... Add required DICOM tags (PatientID, StudyID, SeriesNumber, etc.)
    # ... Extract relevant metadata from the NIfTI header and map it to DICOM tags

    # Iterate through slices and create DICOM instances
    for i in range(nifti_data.shape[2]):  # Assuming 3D volume
        dicom_instance = pydicom.Dataset()
        dicom_instance.PixelData = nifti_data[:, :, i].tobytes()
        # ... Add other required DICOM tags for the instance (InstanceNumber, etc.)
        # ... Set metadata from the main DICOM dataset (dicom_ds)

        # Save the DICOM instance
        instance_filename = f"{output_path}/slice_{i:04d}.dcm"
        dicom_instance.save_as(instance_filename)

def run_pipeline(input_pattern, output_dir, pipeline_args):
    """Creates and runs the Beam pipeline."""

    with beam.Pipeline(options=PipelineOptions(pipeline_args)) as pipeline:
        (pipeline
         | 'ReadNIfTI' >> beam.io.ReadFromText(input_pattern) # Read a file containing a list of NIfTI file paths
         | 'ConvertToDICOM' >> beam.Map(lambda path: nifti_to_dicom(path, output_dir))
        )

if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument('--input_pattern', required=True, help='GCS or local path pattern to NIfTI files (e.g., "gs://my-bucket/nifti/*.nii.gz")')
    parser.add_argument('--output_dir', required=True, help='GCS or local directory to store DICOM files')
    known_args, pipeline_args = parser.parse_known_args()

    run_pipeline(known_args.input_pattern, known_args.output_dir, pipeline_args)


usage: ipykernel_launcher.py [-h] --input_pattern INPUT_PATTERN --output_dir
                             OUTPUT_DIR
ipykernel_launcher.py: error: the following arguments are required: --input_pattern, --output_dir


SystemExit: 2

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
!python -m venv env

In [8]:
!source env/bin/activate
!pip install --upgrade pip
!pip install --target ./package .
!zip -r nifti-to-dicom.zip ./package ./nifti_to_dicom.py

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
	zip warning: name not matched: ./package
	zip warning: name not matched: ./nifti_to_dicom.py

zip error: Nothing to do! (try: zip -r nifti-to-dicom.zip . -i ./package ./nifti_to_dicom.py)


In [9]:
!python nifti_to_dicom.py \
    --input_pattern "gs://your-bucket/nifti-files/*.nii.gz" \
    --output_dir "gs://your-bucket/dicom-output/" \
    --runner DataflowRunner \
    --project your-project-id \
    --region us-central1 \
    --temp_location gs://your-bucket/temp/ 


python: can't open file '/home/jupyter/nifti_to_dicom.py': [Errno 2] No such file or directory
